In [14]:
pip install pip install contexto -f https://ucd-dnp.github.io/ConTexto/contexto_stable.html

Looking in links: https://ucd-dnp.github.io/ConTexto/contexto_stable.html
Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import sys
import pandas as pd
import numpy as np
import shutil
from scipy import sparse, triu
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering

from contexto.limpieza import *
from contexto.exploracion import *
from contexto.vectorizacion import *
from contexto.stemming import Stemmer, stem_texto
from contexto.comparacion import *
from nltk.corpus import stopwords

import plotly.graph_objects as go

In [4]:
datos = pd.read_csv("/Users/juancamiloperdomo/Documents/CNC/Evaluación_Contratos_Paz/Base_datos/Base-códigos.csv")

In [5]:
datos.head()

,Contrato Plan,Sector,Proyecto,Código_proyecto,Estado
0,Bolívar y Sucre - Contrato Paz,Salud y Proteccion Social,CONSTRUCCION Y DOTACION DE INFRAESTRUCTURA SED...,BSSAL24,En Gestión
1,Bolívar y Sucre - Contrato Paz,Vivienda Ciudad y Territorio - Agua,E Y D PARA LA CONSTRUCCION DE UN RELLENO SANIT...,BSAGUA31,En Gestión
2,Bolívar y Sucre - Contrato Paz,Planeacion,ESTUDIOS Y DISEÑOS PARA LA PAVIMENTACION O REH...,BSPLA39,En Gestión
3,Bolívar y Sucre - Contrato Paz,Comercio Industria y Turismo,IMPLEMENTACION DEL PROGRAMA TURISMO Y PAZ EN L...,BSCIT8,En Gestión
4,Bolívar y Sucre - Contrato Paz,Salud y Proteccion Social,(MEJORAMIENTO) CONSTRUCCION Y DOTACION DE INFR...,BSSAL23,En Gestión


In [6]:
mis_stopwords = cargar_stopwords('mis_stopwords.txt')
stopwords_esp = stopwords.words('spanish')
stopwords = lista_stopwords()

In [7]:
lista_palabras = []
lista_expresiones = mis_stopwords[1]

In [8]:
for i in [stopwords, mis_stopwords[0]]:
    lista_palabras += i

In [9]:
lista_palabras = list(set(lista_palabras))
lista_expresiones = list(set(lista_expresiones))

In [10]:
datos['Proyecto limpio'] = datos['Proyecto'].map(lambda x: limpieza_texto(texto=x,
                                                                                                          lista_palabras=lista_palabras,
                                                                                                          lista_expresiones=lista_expresiones,
                                                                                                          ubicacion_archivo=None,
                                                                                                          n_min=1, 
                                                                                                          quitar_numeros=True, 
                                                                                                          quitar_acentos=True))

In [11]:
stemmer = Stemmer(lenguaje='español')
datos['Proyecto_limpio_stemming'] = datos['Proyecto limpio'].map(lambda x: stem_texto(x,stemmer=stemmer))

In [12]:
shutil.rmtree('out/', ignore_errors=True)
sectores = list(datos['Sector'].unique())
sectores.append('Todos')

In [32]:
def getMatrix_fromList(list):
    data = []    
    for i in range(1, len(list)+1):
        data.append(list)

    return data

print(':::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::')
for Sector in sectores:   

    try:        
        print(Sector[:54])
        nombre_sector = Sector[:54]
        newpath = f'out/{nombre_sector}'
        if not os.path.exists(newpath):
            os.makedirs(newpath)

        if Sector != 'Todos':
            df = datos[datos['Sector']==Sector].copy()
            
        else:
            df = datos.copy()
            
        lista_textos = df['Proyecto limpio'].tolist()
        lista_textos_stem = df['Proyecto_limpio_stemming'].tolist()
        texto_completo = df['Proyecto limpio'].str.cat(sep=' ')

        lista_textos_analisis = lista_textos_stem

        texto = texto_completo
        n_grama = 1
        n_terminos= 150

        dictu = frecuencia_ngramas(texto, n_grama, n_terminos)
        dictu = pd.DataFrame(dictu.items(), columns=['Término', 'Frecuencia'])

        # Graficar y guardar nubes de palabras y bigramas
        texto = texto_completo
        n_grama = 1
        n_terminos=50
        graficar=False
        nube_palabras(texto, n_grama=n_grama, n_terminos=n_terminos*2, 
                      titulo=nombre_sector, ubicacion_archivo=f'./out/{nombre_sector}/nube_bigrama.jpg',
                      hor=1, graficar=graficar)

        grafica_barchart_frecuencias(texto, n_grama=n_grama, dim_figura=(10, 10), titulo=nombre_sector, ascendente=True, 
                                     ubicacion_archivo=f'./out/{nombre_sector}/barchart_bigrama.jpg',
                                     graficar=graficar, n_terminos=n_terminos)
      
    except Exception as e:
        f_sector = "{:<54}".format(nombre_sector)
        error = str(e)        
        print(f'--- Error: {f_sector} : {error}')
        continue

:::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Salud y Proteccion Social
Vivienda Ciudad y Territorio - Agua
Planeacion
Comercio Industria y Turismo
Inclusion Social y Reconciliacion
Transporte
Cultura
Vivienda Ciudad y Territorio - Vivienda
Educacion
Agricultura  y Desarrollo Rural
Ambiente y Desarrollo Sostenible
Deporte y Recreacion
Justicia y Del_Derecho
Interior
Minas y Energia
Trabajo
Tecnologías de la Información y las Comunicaciones
Defensa y Policia
inclusion Social y Reconciliacion
Todos
